In [1]:
import os 

In [2]:
%pwd

'd:\\Machine-learning-project-with-Mlflow\\research'

In [3]:
os.chdir('../')

In [4]:
%pwd

'd:\\Machine-learning-project-with-Mlflow'

In [5]:
from dataclasses import dataclass
from pathlib import Path

@dataclass(frozen=True)
class DataIngestionConfig:
    root_dir: Path
    source_url: str
    local_data_file: Path
    unzip_dir: Path

In [6]:
file_path = r"D:\Machine-learning-project-with-Mlflow\src\mlProject\constants\__init__.py"

with open(file_path, 'r') as f:
    content = f.read()
    print(content)

from pathlib import Path

CONFIG_FILE_PATH = Path("config/config.yaml")
PARAMS_FILE_PATH = Path("params.yaml")
SCHEMA_FILE_PATH = Path("config/schema.yaml")


In [7]:
file_path = r"D:\Machine-learning-project-with-Mlflow\src\mlProject\entity\config_entity.py"

with open(file_path, 'r') as f:
    content = f.read()
    print(content)

from dataclasses import dataclass
from pathlib import Path

@dataclass(frozen=True)
class DataIngestionConfig:
    root_dir: Path
    source_url: str
    local_data_file: Path
    unzip_dir: Path


In [8]:
from pathlib import Path
from mlProject.constants import CONFIG_FILE_PATH, PARAMS_FILE_PATH, SCHEMA_FILE_PATH
from mlProject.utils.common import read_yaml, create_directories
from mlProject.entity.config_entity import DataIngestionConfig  

In [9]:
class ConfigurationManager:
    def __init__(
        self,
        config_file_path: Path = CONFIG_FILE_PATH,
        params_file_path: Path = PARAMS_FILE_PATH,    
        schema_file_path: Path = SCHEMA_FILE_PATH):
        
        self.config = read_yaml(config_file_path)
        self.params = read_yaml(params_file_path)
        self.schema = read_yaml(schema_file_path)
        
        create_directories([self.config.artifacts_root])
    
    def get_data_ingestion_config(self) -> DataIngestionConfig:
        config = self.config.data_ingestion
        
        create_directories([config.root_dir])
        
        data_ingestion_config = DataIngestionConfig(
            root_dir=Path(config.root_dir),
            source_url=config.source_url,
            local_data_file=Path(config.local_data_file),
            unzip_dir=Path(config.unzip_dir)
        )
        
        return data_ingestion_config

In [10]:
import os
import urllib.request as request
import zipfile
from mlProject import logger
from mlProject.utils.common import get_size


In [12]:
class DataIngestion:
    def __init__(self, config: DataIngestionConfig):
        self.config = config


    def download_data(self):
        if not os.path.exists(self.config.local_data_file):
            filename, headers = request.urlretrieve(
               url = self.config.source_url,
                filename = self.config.local_data_file
                )
            logger.info(f"File: {filename} downloaded with following info: \n{headers}")
        else:
            logger.info(f"File already exists of size: {get_size(Path(self.config.local_data_file))}")
    
        
    def extract_zip_file(self):
        """
        zip_file_path: str
        Extracts the zip file into the unzip_dir
        function does not return anything
        """
        unzip_path = self.config.unzip_dir
        os.makedirs(unzip_path, exist_ok=True)  
        with zipfile.ZipFile(self.config.local_data_file, 'r') as zip_ref:
            zip_ref.extractall(unzip_path)

In [15]:
try:
    config = ConfigurationManager()
    data_ingestion_config = config.get_data_ingestion_config()
    data_ingestion = DataIngestion(config=data_ingestion_config)
    data_ingestion.download_data()
    data_ingestion.extract_zip_file()
except Exception as e:
    raise e

[2025-12-23 18:05:19,775 : INFO : common : YAML file 'config\config.yaml' read successfully.]
[2025-12-23 18:05:19,780 : INFO : common : YAML file 'params.yaml' read successfully.]
[2025-12-23 18:05:19,785 : INFO : common : YAML file 'config\schema.yaml' read successfully.]
[2025-12-23 18:05:19,788 : INFO : common : Created directory at: artifacts_root]
[2025-12-23 18:05:19,794 : INFO : common : Created directory at: artifacts/data_ingestion]
[2025-12-23 18:05:21,599 : INFO : 1726274682 : File: artifacts\data_ingestion\data.zip downloaded with following info: 
Connection: close
Content-Length: 23329
Cache-Control: max-age=300
Content-Security-Policy: default-src 'none'; style-src 'unsafe-inline'; sandbox
Content-Type: application/zip
ETag: "c69888a4ae59bc5a893392785a938ccd4937981c06ba8a9d6a21aa52b4ab5b6e"
Strict-Transport-Security: max-age=31536000
X-Content-Type-Options: nosniff
X-Frame-Options: deny
X-XSS-Protection: 1; mode=block
X-GitHub-Request-Id: DDCA:36B367:A3598:2708C8:694AAFA